In [1]:
import pickle as pkl
import numpy as np

In [2]:
data, _ = pkl.load(open("traditional_drawings_abstract.p", "rb"))
data2, _ = pkl.load(open("traditional_paintings_abstract.p", "rb"))


In [3]:
data = data + data2

In [4]:
try:
    user_id_dict, user_count = pkl.load(open('user_id2.p', 'rb'))
    item_id_dict, item_count = pkl.load(open('item_id2.p', 'rb'))
    id_img_dict = pkl.load(open('id_img2.p', 'rb'))
except FileNotFoundError:
    user_id_dict = {}; item_id_dict = {}; id_img_dict = {}
    user_count = 0; item_count = 0

In [5]:
artists = [entry[0] for entry in data]
favorite_lists = [[pair[0] for pair in entry[-1]] for entry in data]
favorite_lists_flattened = [item for sublist in favorite_lists for item in sublist]
img_ids = [entry[1] for entry in data]
img_srcs = [entry[3] for entry in data]
times = [entry[2] for entry in data]



In [8]:
converted_img_ids = [item_id_dict[item] for item in img_ids]
num_favorites = [len(item) for item in favorite_lists]

img_nfavs_dict = dict(zip(converted_img_ids, num_favorites))
pkl.dump(img_nfavs_dict, open('img_nfavs.p', 'wb'))

In [9]:
np.mean(num_favorites)

47.072922736277825

In [13]:
rev = dict(zip(list(user_id_dict.values()), list(user_id_dict.keys()))) #map user_id to user 

In [12]:
try:
    favoriter_dict = pkl.load(open('favorited_dict2.p', 'rb'))
except FileNotFoundError:
    favoriter_dict = {}

In [7]:
for artist, img_id, img_src, favorite_list in zip(artists,img_ids,img_srcs, favorite_lists):
    
    if img_id in item_id_dict: # For some reason, there's a duplicate
        continue # So just keep going 
    else:
        item_id = item_count;
        item_id_dict[img_id] = item_id; item_count += 1
    
    artist = artist.username
    try:
        artist_id = user_id_dict[artist]
    except:
        artist_id = user_count;
        user_id_dict[artist] = artist_id; user_count += 1
    
    
    id_img_dict[item_id] = (img_src, artist_id)
    
    fav_ids = []
    for favoriter in favorite_list:
        favoriter = favoriter.username
        try:
            fav_id = user_id_dict[favoriter]
        except:
            fav_id = user_count;
            user_id_dict[favoriter] = fav_id; user_count += 1
        finally:
            try:
                favoriter_dict[fav_id].extend([item_id])
            except KeyError:
                favoriter_dict[fav_id] = [item_id]

In [8]:
pkl.dump(favoriter_dict, open('favorited_dict2.p', 'wb'))
pkl.dump((user_id_dict, user_count), open('user_id2.p', 'wb'))
pkl.dump((item_id_dict, item_count), open('item_id2.p', 'wb'))
pkl.dump(id_img_dict,  open('id_img2.p', 'wb'))


max_item = max(item_id_dict.values())
max_user = max(user_id_dict.values())
pkl.dump((max_item+1, max_user+1), open('useful_stats2.p', 'wb'))

users_with_favorites = list(favoriter_dict.keys())
np.random.shuffle(users_with_favorites)


writer = tf.python_io.TFRecordWriter("test2.tfrecords")

from tqdm import tqdm

for user in tqdm(users_with_favorites):
    favorited_items = favoriter_dict[user]
    ''' construct the Example proto object '''
    example = tf.train.Example(
        ''' Example contains a Features proto object '''
        features=tf.train.Features(
          ''' Features contains a map of string to Feature proto objects '''
          feature={
            ''' A Feature contains one of either a int64_list,
             float_list, or bytes_list '''
            'user': tf.train.Feature(
                int64_list=tf.train.Int64List(value=[user])),

            'favorited': tf.train.Feature(
                int64_list=tf.train.Int64List(value=favorited_items)),
    }))
    serialized = example.SerializeToString()
    ''' write the serialized object to disk '''
    writer.write(serialized)



In [9]:
g5 = sum(map(lambda x: len(x)>=5, favoriter_dict.values()))
sum_g5 = sum(map(lambda x: len(x) if len(x)>5 else 0, favoriter_dict.values()))
num_users = len(user_id_dict)
num_items = len(item_id_dict)
num_ratings = sum(map(lambda x: len(x), favoriter_dict.values()))

print ("""There are {} users and {} items in the dataset, for a total of {} favorites. 
Of those users, {} of them have favorited more than 5 objects, accounting for {} ratings.""".format(
    num_users, num_items, num_ratings, g5, sum_g5))

There are 686975 users and 66431 items in the dataset, for a total of 3203375 favorites. 
Of those users, 122010 of them have favorited more than 5 objects, accounting for 2202460 ratings.


In [47]:
artist_dict = {}
item_to_artist = {}

In [48]:
for artist, img_id, time in zip(artists,img_ids, times):
    artist = artist.username
    artist_id = user_id_dict[artist]
    
    item_id = item_id_dict[img_id]
    try:
        artist_dict[artist_id].extend([(item_id, time)])
    except:
        artist_dict[artist_id] = [(item_id, time)]
        
    item_to_artist[item_id] = artist_id

In [62]:
import operator

for artist in artist_dict:
    artist_dict[artist].sort(key=operator.itemgetter(1))
    artist_dict[artist] = [i[0] for i in artist_dict[artist]]

In [23]:
g5 = sum(map(lambda x: len(x)>=5, artist_dict.values()))
g20 = sum(map(lambda x: len(x)>=20, artist_dict.values()))
g50 = sum(map(lambda x: len(x)>=50, artist_dict.values()))

total50 = sum(map(lambda x: len(x) if len(x)>=50 else 0, artist_dict.values()))

total = sum(map(lambda x: len(x), artist_dict.values()))

num_artists = len(artist_dict)


print('There are {} total artists, of whom {}, {}, {} have more than 5, 20, 50 respectively'.format(num_artists, g5, g20, g50))
print ('The {} artists with 50+ uploads have posted {} of the {} pieces of artwork'.format(g50, total50, total))

There are 18014 total artists, of whom 2130, 480, 179 have more than 5, 20, 50 respectively
The 179 artists with 50+ uploads have posted 23296 of the 68648 pieces of artwork


In [64]:
pkl.dump(artist_dict, open('artist_dict.p', 'wb'))
pkl.dump(item_to_artist, open('item_to_artist.p', 'wb'))